# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff
using PseudoPotentialData

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice = T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    pseudopotentials = PseudoFamily("cp2k.nc.sr.lda.v0_1.semicore.gth")
    atoms     = [ElementPsp(:He, pseudopotentials)]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions;
                      functionals=[:lda_x, :lda_c_vwn],
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770818573708                   -0.52    9.0    167ms
  2   -2.772143892869       -2.88       -1.32    1.0    103ms
  3   -2.772170384335       -4.58       -2.52    1.0    140ms
  4   -2.772170672610       -6.54       -3.32    1.0    420ms
  5   -2.772170721499       -7.31       -3.91    2.0    117ms
  6   -2.772170722972       -8.83       -5.39    1.0    137ms
  7   -2.772170723013      -10.38       -5.36    3.0    146ms
  8   -2.772170723015      -11.82       -5.88    1.0    104ms
  9   -2.772170723015      -12.83       -6.50    1.0    107ms
 10   -2.772170723015   +  -14.88       -7.32    1.0    106ms
 11   -2.772170723015   +    -Inf       -7.39    2.0    121ms
 12   -2.772170723015      -13.91       -7.84    1.0    109ms
 13   -2.772170723015   +  -14.12       -8.62    1.0    118ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ----

1.7735580679044824

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem. This leads to a density-functional perturbation
theory problem, which is automatically set up and solved in the background.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770757074237                   -0.52    9.0    175ms
  2   -2.772059113465       -2.89       -1.33    1.0    105ms
  3   -2.772082825038       -4.63       -2.40    1.0    100ms
  4   -2.772083330661       -6.30       -3.10    1.0    144ms
  5   -2.772083417638       -7.06       -4.78    2.0    451ms
  6   -2.772083417794       -9.81       -4.81    2.0    122ms
  7   -2.772083417810      -10.81       -5.81    1.0    127ms
  8   -2.772083417811      -12.04       -6.15    2.0    119ms
  9   -2.772083417811      -13.78       -6.87    1.0    107ms
 10   -2.772083417811      -14.24       -8.19    1.0    108ms
Solving response problem
Iter  Restart  Krydim  log10(res)  avg(CG)  Δtime   Comment
----  -------  ------  ----------  -------  ------  ---------------
                                      13.0  79.5ms  Non-interacting
   1        0       1       -0.60   